In [1]:
import numpy as np 
import pandas as pd
import glob
import plotly.express as px
import requests


%load_ext autoreload
%autoreload 2

In [2]:
#load in the processed data 
X = pd.read_csv("../processed/caiso_final_X.csv", index_col=0)
y = pd.read_csv("../processed/caiso_final_y.csv")

#filter the columns in y so that it only has the Solar production column (Solar Output (MW)) and the date index
y = y[["Time","Solar Output (MW)"]]
y.head()

,Time,Solar Output (MW)
0,2018-04-10 00:00:00,0.0
1,2018-04-10 01:00:00,0.0
2,2018-04-10 02:00:00,0.0
3,2018-04-10 03:00:00,0.0
4,2018-04-10 04:00:00,0.0


In [3]:
#now cut off the y time index to match the X time index
y = y[y["Time"].isin(X.index)]
y

,Time,Solar Output (MW)
5455,2019-01-01 00:00:00,0.000000
5456,2019-01-01 01:00:00,0.000000
5457,2019-01-01 02:00:00,0.000000
5458,2019-01-01 03:00:00,0.000000
5459,2019-01-01 04:00:00,0.000000
...,...,...
53494,2025-05-24 19:00:00,1789.750000
53495,2025-05-24 20:00:00,-61.583333
53496,2025-05-24 21:00:00,-66.833333
53497,2025-05-24 22:00:00,-63.750000


In [4]:
#now we can build our model out

#first we would like to build a preliminary model to see how well we can predict the Solar Output (MW) using the features in X. 

# Align X to only the times present in y["Time"]
X_aligned = X.loc[y["Time"]]

#we must split the data squentially into train and test sets.
from sklearn.model_selection import train_test_split
#split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_aligned, y["Solar Output (MW)"], test_size=0.2, shuffle=False)

#now build a linear regression model
from sklearn.linear_model import LinearRegression
#initialize the model
model = LinearRegression()
#fit the model to the training data
model.fit(X_train, y_train)
#now we can make predictions on the test set
y_pred = model.predict(X_test)
#now we can evaluate the model
from sklearn.metrics import mean_squared_error, r2_score
#calculate the mean squared error and r2 score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")
#now we can visualize the predictions using plotly
fig = px.scatter(x=y_test, y=y_pred, labels={'x': 'Actual Solar Output (MW)', 'y': 'Predicted Solar Output (MW)'}, title='Actual vs Predicted Solar Output')
fig.add_shape(type="line", x0=min(y_test), y0=min(y_test), x1=max(y_test), y1=max(y_test), line=dict(color="Red", width=2, dash="dash"))
fig.show()

Mean Squared Error: 1220794.6128154031
R2 Score: 0.9753961894262336


In [8]:
#plot top 10 feature importances by magnitude with plotly
importances = model.coef_
feature_names = X_train.columns
# Create a DataFrame for feature importances
feature_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
# Sort the DataFrame by absolute importance
feature_importances['AbsImportance'] = feature_importances['Importance'].abs()
top_features = feature_importances.sort_values(by='AbsImportance', ascending=False).head(10)
# Plot the top 10 feature importances
fig = px.bar(top_features, x='Feature', y='AbsImportance', title='Top 10 Feature Importances', labels={'AbsImportance': 'Absolute Importance'})
fig.update_layout(xaxis_title='Feature', yaxis_title='Absolute Importance')
fig.show()



In [9]:
#now let's use xgboost to build a model
from xgboost import XGBRegressor
#initialize the model
xgb_model = XGBRegressor(objective='reg:squarederror', n_estimators=1000, learning_rate=0.1, max_depth=6)
#fit the model to the training data
xgb_model.fit(X_train, y_train)
#now we can make predictions on the test set
y_pred_xgb = xgb_model.predict(X_test)
#now we can evaluate the model
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)
print(f"XGBoost Mean Squared Error: {mse_xgb}")
print(f"XGBoost R2 Score: {r2_xgb}")
#now we can visualize the predictions using plotly
fig_xgb = px.scatter(x=y_test, y=y_pred_xgb, labels={'x': 'Actual Solar Output (MW)', 'y': 'Predicted Solar Output (MW)'}, title='Actual vs Predicted Solar Output (XGBoost)')
fig_xgb.add_shape(type="line", x0=min(y_test), y0=min(y_test), x1=max(y_test), y1=max(y_test), line=dict(color="Red", width=2, dash="dash"))
fig_xgb.show()

XGBoost Mean Squared Error: 2217705.6528430255
XGBoost R2 Score: 0.955304512963827


In [11]:
#now let's try a neural network model using keras since this data is reasonably big
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

# Reduce model complexity: fewer layers and units
nn_model = Sequential()
nn_model.add(Dense(32, activation='relu', input_shape=(X_train.shape[1],)))
nn_model.add(Dropout(0.1))
nn_model.add(Dense(16, activation='relu'))
nn_model.add(Dense(1))  # Output layer for regression

# Compile the model
nn_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# Add early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Fit the model to the training data
history = nn_model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=64,
    validation_split=0.2,
    verbose=1,
    callbacks=[early_stop]
)

# Make predictions on the test set
y_pred_nn = nn_model.predict(X_test)

# Evaluate the model
mse_nn = mean_squared_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)
print(f"Neural Network Mean Squared Error: {mse_nn}")
print(f"Neural Network R2 Score: {r2_nn}")

# Visualize the predictions using plotly
fig_nn = px.scatter(
    x=y_test, y=y_pred_nn.flatten(),
    labels={'x': 'Actual Solar Output (MW)', 'y': 'Predicted Solar Output (MW)'},
    title='Actual vs Predicted Solar Output (Neural Network)'
)
fig_nn.add_shape(
    type="line",
    x0=min(y_test), y0=min(y_test),
    x1=max(y_test), y1=max(y_test),
    line=dict(color="Red", width=2, dash="dash")
)
fig_nn.show()


Epoch 1/100
481/481 [==============================] - 2s 3ms/step - loss: 1386131.2500 - mae: 687.0919 - val_loss: 1242613.5000 - val_mae: 665.0017
Epoch 2/100
481/481 [==============================] - 1s 2ms/step - loss: 544820.9375 - mae: 434.2174 - val_loss: 761476.2500 - val_mae: 498.0594
Epoch 3/100
481/481 [==============================] - 1s 2ms/step - loss: 425560.0938 - mae: 375.6873 - val_loss: 800203.0000 - val_mae: 534.4766
Epoch 4/100
481/481 [==============================] - 1s 2ms/step - loss: 374553.2500 - mae: 350.1846 - val_loss: 990316.8750 - val_mae: 609.5272
Epoch 5/100
481/481 [==============================] - 1s 2ms/step - loss: 347778.1562 - mae: 335.6537 - val_loss: 747963.6250 - val_mae: 521.4233
Epoch 6/100
481/481 [==============================] - 1s 2ms/step - loss: 319018.9062 - mae: 317.8488 - val_loss: 956138.8750 - val_mae: 604.0109
Epoch 7/100
481/481 [==============================] - 1s 2ms/step - loss: 290799.9375 - mae: 304.1198 - val_loss: 9

In [9]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Ensure y_train index matches X_train index
y_train_aligned = y_train.copy()
y_train_aligned.index = X_train.index

# Align exogenous variables for the test period
X_test_exog = X_test

# Fit SARIMAX with exogenous variables (order and seasonal_order may need tuning)
sarimax_model = SARIMAX(
    y_train_aligned, 
    exog=X_train, 
    order=(2, 1, 2), 
    seasonal_order=(1, 0, 1, 24),  # daily seasonality for hourly data
    enforce_stationarity=False, 
    enforce_invertibility=False
)
sarimax_fit = sarimax_model.fit(disp=False)

# Forecast using exogenous variables for the test set
y_pred_sarimax = sarimax_fit.forecast(steps=len(y_test), exog=X_test_exog)

# Evaluate
mse_sarimax = mean_squared_error(y_test, y_pred_sarimax)
r2_sarimax = r2_score(y_test, y_pred_sarimax)
print(f"SARIMAX Mean Squared Error: {mse_sarimax}")
print(f"SARIMAX R2 Score: {r2_sarimax}")

# Plot
fig_sarimax = px.scatter(
    x=y_test, y=y_pred_sarimax,
    labels={'x': 'Actual Solar Output (MW)', 'y': 'Predicted Solar Output (MW)'},
    title='Actual vs Predicted Solar Output (SARIMAX)'
)
fig_sarimax.add_shape(
    type="line",
    x0=min(y_test), y0=min(y_test),
    x1=max(y_test), y1=max(y_test),
    line=dict(color="Red", width=2, dash="dash")
)
fig_sarimax.show()


/Users/gordondoore/opt/anaconda3/envs/cs444/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/Users/gordondoore/opt/anaconda3/envs/cs444/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/Users/gordondoore/opt/anaconda3/envs/cs444/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

/Users/gordondoore/opt/anaconda3/envs/cs444/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/Users/gordondoore/opt/anaconda3/envs/cs444/lib/python3.10/s

SARIMAX Mean Squared Error: 2158705.435778142
SARIMAX R2 Score: 0.9564935992763295


In [10]:
#plot predictions from the linear regression as a time series
import plotly.graph_objects as go
# Create a time series plot for the linear regression predictions
fig_time_series = go.Figure()
# Add actual values
fig_time_series.add_trace(go.Scatter
    (x=y_test.index, y=y_test, mode='lines', name='Actual Solar Output (MW)', line=dict(color='blue')))
# Add predicted values
fig_time_series.add_trace(go.Scatter
    (x=y_test.index, y=y_pred, mode='lines', name='Predicted Solar Output (MW)', line=dict(color='red')))
# Add layout details
fig_time_series.update_layout(
    title='Time Series of Actual vs Predicted Solar Output (Linear Regression)',
    xaxis_title='Time',
    yaxis_title='Solar Output (MW)',
    legend=dict(x=0, y=1, traceorder='normal')
)
fig_time_series.show()

In [12]:
#show to predictions of the last 7 days of the test set
last_7_days = y_test.index[-7*24:]
# Create a time series plot for the last 7 days of predictions
fig_last_7_days = go.Figure()
# Add actual values for the last 7 days
fig_last_7_days.add_trace(go.Scatter
    (x=y_test.index[-7*24:], y=y_test[-7*24:], mode='lines', name='Actual Solar Output (MW)', line=dict(color='blue')))
# Add predicted values for the last 7 days
fig_last_7_days.add_trace(go.Scatter
    (x=y_test.index[-7*24:], y=y_pred[-7*24:], mode='lines', name='Predicted Solar Output (MW)', line=dict(color='red')))
# Add layout details
fig_last_7_days.update_layout(
    title='Time Series of Actual vs Predicted Solar Output (Last 7 Days)',
    xaxis_title='Time',
    yaxis_title='Solar Output (MW)',
    legend=dict(x=0, y=1, traceorder='normal')
)
fig_last_7_days.show()

In [19]:
#ok lets finally try a polynomial regression model with degree 2
from sklearn.preprocessing import PolynomialFeatures
# Create polynomial features
poly = PolynomialFeatures(degree=2)
# Transform the training and test data
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)
# Fit a linear regression model on the polynomial features
poly_model = LinearRegression()
# Fit the model to the polynomial features
poly_model.fit(X_train_poly, y_train)
# Make predictions on the test set
y_pred_poly = poly_model.predict(X_test_poly)
# Evaluate the polynomial regression model
mse_poly = mean_squared_error(y_test, y_pred_poly)
r2_poly = r2_score(y_test, y_pred_poly)
print(f"Polynomial Regression Mean Squared Error: {mse_poly}")
print(f"Polynomial Regression R2 Score: {r2_poly}")
# Visualize the predictions using plotly
fig_poly = px.scatter(
    x=y_test, y=y_pred_poly,
    labels={'x': 'Actual Solar Output (MW)', 'y': 'Predicted Solar Output (MW)'},
    title='Actual vs Predicted Solar Output (Polynomial Regression)'
)
fig_poly.add_shape(
    type="line",
    x0=min(y_test), y0=min(y_test),
    x1=max(y_test), y1=max(y_test),
    line=dict(color="Red", width=2, dash="dash")
)
fig_poly.show()
# Plot polynomial regression predictions as a time series
fig_poly_time_series = go.Figure()
# Add actual values
fig_poly_time_series.add_trace(go.Scatter(
    x=y_test.index, y=y_test, mode='lines', name='Actual Solar Output (MW)', line=dict(color='blue')))
# Add predicted values
fig_poly_time_series.add_trace(go.Scatter(
    x=y_test.index, y=y_pred_poly, mode='lines', name='Predicted Solar Output (MW)', line=dict(color='red')))
# Add layout details
fig_poly_time_series.update_layout(
    title='Time Series of Actual vs Predicted Solar Output (Polynomial Regression)',
    xaxis_title='Time',
    yaxis_title='Solar Output (MW)',
    legend=dict(x=0, y=1, traceorder='normal')
)
fig_poly_time_series.show()
# Show predictions of the last 7 days of the polynomial regression model
last_7_days_poly = y_test.index[-7*24:]
# Create a time series plot for the last 7 days of polynomial regression predictions
fig_last_7_days_poly = go.Figure()
# Add actual values for the last 7 days
fig_last_7_days_poly.add_trace(go.Scatter(
    x=y_test.index[-7*24:], y=y_test[-7*24:], mode='lines', name='Actual Solar Output (MW)', line=dict(color='blue')))
# Add predicted values for the last 7 days
fig_last_7_days_poly.add_trace(go.Scatter(
    x=y_test.index[-7*24:], y=y_pred_poly[-7*24:], mode='lines', name='Predicted Solar Output (MW)', line=dict(color='red')))
# Add layout details
fig_last_7_days_poly.update_layout(
    title='Time Series of Actual vs Predicted Solar Output (Last 7 Days, Polynomial Regression)',
    xaxis_title='Time',
    yaxis_title='Solar Output (MW)',
    legend=dict(x=0, y=1, traceorder='normal')
)
fig_last_7_days_poly.show()


Polynomial Regression Mean Squared Error: 801300.9870553876
Polynomial Regression R2 Score: 0.9838506350772503


In [22]:
#Ok now let's save the polynomial regression model 
import joblib
# Save the polynomial regression model
joblib.dump(poly_model, '../../models/polynomial_regression_model_renewable.pkl')
# Save the polynomial features transformer
joblib.dump(poly, '../../models/polynomial_features_transformer_renewable.pkl')


['../../models/polynomial_features_transformer_renewable.pkl']